In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
from ampl_utils import ampl_license
from amplpy import AMPL, ampl_notebook

from models import MixedStrategyMasterProblem, to_set_ids, MathematicalModel, values_to_list, AttackGenerationPricingProblem

from dataloader import Network

from itertools import product

solver='cplex'
ampl_license_uuid = ampl_license()

ampl = ampl_notebook(
    modules=[solver],
    license_uuid=ampl_license_uuid)

Licensed to AMPL Academic Community Edition License for <s207399@student.pg.edu.pl>.


### Master Problem

$$
\begin{aligned}
\mathbb P[\mathcal S, \mathcal A]: \quad & \max y \quad & \\
[x] \quad & \sum_{s\in\mathcal S}q_s = 1 \quad & \\
[p_a\geq 0]\quad & y \leq \sum_{s\in\mathcal S}V(s, a)q_s \quad & a\in\mathcal A \\
 & y \in\mathbb R; q_s\in\mathbb{R}_+\quad & s\in\mathcal S
\end{aligned}
$$

Here's a simple test of the master problem

In [6]:
network = Network('dognet')
attack_set = [
    [1, 2, 3], [5, 7, 8], [7, 2, 3], [4, 9, 8], [1, 5, 4], [9, 2, 1]
]

placement_set = [
    [1, 9], [5, 4], [2, 7], [3, 8], [8, 9]
]
mp = MixedStrategyMasterProblem(network, placement_set, attack_set)
mp.solve()
mp.report()

CPLEX 22.1.2: optimal solution; objective 3.25
6 simplex iterations


{'V*': 3.25, 'q': [0, 0.375, 0, 0.375, 0.25], 'p': [0, 0.25, 0.75, 0, 0, 0]}

### Attack Generation Pricing Problem

The attack generation pricing problem is denoted by

$$
\begin{aligned}
\min\sum_{s\in\mathcal S'}q^*_s\cdot F_s \quad & \quad &  \\
\sum_{v\in\mathcal V} a_v = K \quad&\quad & \text{Must be a $K$-node attack}\\
z_v^s = 1-a_v \quad & v\in\mathcal V, s\in\mathcal S' \quad& \text{$v$ survives attack $a$ given $s$} \\
z_{\beta(e)}^s \geq z_{\alpha(e)^s} - a_{\beta(e)} \quad & e\in\mathcal E, s\in\mathcal S'\quad & \\
z_{\alpha(e)}^s \geq z_{\beta(e)^s} - a_{\alpha(e)} \quad & e\in\mathcal E, s\in\mathcal S'\quad & \\
F_s = \sum_{v\in\mathcal V}z_v^s \quad & s\in\mathcal S' \quad & \text{The number of surviving nodes per placement} \\
a_v, z_v^s\in\mathbb B\quad & s\in\mathcal S', v\in\mathcal V \quad & \\
F_s \in\mathbb Z_+ \quad & s\in\mathcal S' \quad & 
\end{aligned}
$$

where $\mathcal S'=\{s\in\mathcal S : q^*(s) > 0 \}$ is the set of placements with non-zero probabilities of being used.

In [13]:
agpp = AttackGenerationPricingProblem(network, [[3], [5]], [4.0, 0.6], 2)
agpp.solve()
agpp.report()

CPLEX 22.1.2: optimal solution; objective 0
0 simplex iterations


{'a*': [0, 0, 1, 0, 1, 0, 0, 0, 0]}

In [166]:
%%writefile models/controller_placement_pricing_problem.mod

# vertices in the graph
set VERTICES;
# attacks. These are simply indices
set ATTACKS;
# Nodes affected by the attack
set V_A {ATTACKS} within VERTICES;
# vertices exceeding BCC {{v, w} in V^|2| : d(v, w) > BCC}
set U within {VERTICES, VERTICES};
# C(a) is the set of components resulting from an attack a
## Component Ids hold the indices of components resulting from attack a
set COMPONENT_IDS {ATTACKS};
set C_A {a in ATTACKS, COMPONENT_IDS[a]} within VERTICES;

# Number of surviving nodes given attack
var Y {ATTACKS} integer >= 0;
# Whether node v is a controller
var s {VERTICES} binary;
# The number of surviving nodes given attack a
var y {VERTICES, ATTACKS} binary;

# Total number of controller nodes
param M integer;
# Attack probability
param p_star {ATTACKS};

maximize OperatorPayoff:
    sum {a in ATTACKS} (p_star[a] * Y[a]);

s.t. NumberOfControllers:
    sum {v in VERTICES} s[v] = M;

s.t. ZeroAttackedNodes {a in ATTACKS, v in V_A[a]}:
    y[v, a] = 0;

s.t. ZeroComponentsWithoutControllers {a in ATTACKS, c_id in COMPONENT_IDS[a]}:
    sum {v in C_A[a, c_id]} y[v, a] <= card(C_A[a, c_id]) * sum {v in C_A[a, c_id]} s[v];

s.t. NumberOfSurvivingNodes {a in ATTACKS}:
    Y[a] = sum {v in VERTICES} y[v, a];

Overwriting models/controller_placement_pricing_problem.mod


In [167]:
%%writefile models/controller_placement_pricing_problem_delay.mod

# vertices in the graph
set VERTICES;
# attacks. These are simply indices
set ATTACKS;
# Nodes affected by the attack
set V_A {ATTACKS} within VERTICES;
# vertices exceeding BCC {{v, w} in V^|2| : d(v, w) > BCC}
set U within VERTICES cross VERTICES;
# V_C is the set of components resulting from an attack a
set V_C {ATTACKS} within VERTICES;
# The nodes within the SC-delay constraints of node v
set W_V {VERTICES} within VERTICES;

# Number of surviving nodes given attack
var Y {ATTACKS};
# Whether node v is a controller
var s {VERTICES} binary;
# The number of surviving nodes given attack a
var y {VERTICES, ATTACKS} binary;

# Total number of controller nodes
param M integer;
# Attack probability
param p_star {ATTACKS};

maximize OperatorPayoff:
    sum {a in ATTACKS} (p_star[a] * Y[a]);

s.t. NumberOfControllers:
    sum {v in VERTICES} s[v] = M;

s.t. ZeroAttackedNodes {a in ATTACKS, v in V_A[a]}:
    y[v, a] = 0;

Overwriting models/controller_placement_pricing_problem_delay.mod


In [174]:
class ControllerPlacementPricingProblem(MathematicalModel):
    def __init__(self, network, attacks, p_star, M, eps=1e-9):
        self._model_file = 'models/controller_placement_pricing_problem.mod'
        super(ControllerPlacementPricingProblem, self).__init__(self._model_file)

        self._name = 'ControllerPlacementPricingProblem'
        self.network = network
        self.attacks = attacks
        self.p_star = p_star
        self.M = M
        self.eps = eps

    def report(self):
        if not self._solved:
            self.solve()
        var_s = self.ampl.get_variables()['s']
        s_star = values_to_list(var_s)
        # Y_star = values_to_list(self.ampl.get_variables()['Y'])

        V_star = self.ampl.get_objectives()['OperatorPayoff'].value()
        print(V_star)
        return {
            's*': s_star,
            'V*': V_star,
        }

    def load_data(self):
        self._load_model()
        vertex_list = list(self.network.nodes)

        # Key => attack set pair
        attack_dict = to_set_ids(self.attacks)

        self.ampl.set['VERTICES'] = vertex_list
        self.ampl.set['ATTACKS'] = attack_dict.keys()

        
        V_A = {}
        for i, a in attack_dict.items():
            # vs = self.network.remaining_nodes(a)
            V_A[i] = a
        self.ampl.set['V_A'] = V_A

        # if self.has_delay:
        #     # Pair of nodes that exceed the BSC delay from each other.
        #     U = [
        #         (v, w) for v, w in product(vertex_list, vertex_list)
        #         if v < w and self.network.delays[v - 1, w - 1] > self.bcc
        #     ]
    
        #     self.ampl.set['U'] = U
        #     W_V = {}
    
        #     # The nodes w that are reachable from v within BSC delay.
        #     for v in vertex_list:
        #         within_delay = [
        #             w for w in vertex_list if self.network.delays[v - 1, w - 1] <= self.bsc
        #         ]
        #         W_V[v] = within_delay
    
        #     self.ampl.set['W_V'] = W_V
        
        # Components after the attack. c in C(a) contains the list of vertices in each component
        component_ids = {}
        C_A = {}
        for a, attack in attack_dict.items():
            components = self.network.attack(attack)
            component_ids[a] = list(range(len(components)))
            for i, c in enumerate(components):
                C_A[(a, i)] = list(c)

        print(component_ids)
        self.ampl.set['COMPONENT_IDS'] = component_ids
        self.ampl.set['C_A'] = C_A

        self.ampl.param['M'] = self.M
        self.ampl.param['p_star'] = self.p_star

In [175]:
cppp = ControllerPlacementPricingProblem(network, [[1, 2], [5, 6], [5, 9]], [0.4, 0.5, 0.1], 3)
cppp.report()

{0: [0], 1: [0, 1], 2: [0, 1]}
CPLEX 22.1.2: optimal solution; objective 7
3 simplex iterations
absmipgap=8.88178e-16, relmipgap=0
7.000000000000001


{'s*': [0, 0, 1, 0, 0, 1, 0, 0, 1], 'V*': 7.000000000000001}